In [ ]:
import pandas as pd
import numpy as np
import polars as pl

from statsmodels.tsa.seasonal import seasonal_decompose
from factorlib.utils.system import get_data_dir

In [ ]:
raw_data_dir = get_data_dir() / 'raw'
ohclv_raw = pl.scan_csv(raw_data_dir / 'ohclv_daily.csv', try_parse_dates=True).collect(streaming=True)
ohclv_raw

In [ ]:
closes_raw = ohclv_raw.lazy().select(pl.col('date_index'), pl.col('ticker'), pl.col('prccd')).collect(streaming=True)
closes_raw = closes_raw.to_pandas().set_index('date_index')
closes_raw

In [ ]:
returns = closes_raw
returns['ret'] = closes_raw.groupby('ticker').pct_change().dropna()

In [ ]:
decomposition = seasonal_decompose(returns['ret'], model='additive')
seasonal_decomposition = returns
seasonal_decomposition['trend'] = decomposition.trend
seasonal_decomposition['seasonality'] = decomposition.seasonal
seasonal_decomposition['resid'] = decomposition.resid

In [ ]:
seasonal_decomposition_factor = seasonal_decomposition
seasonal_decomposition_factor = seasonal_decomposition_factor.drop(columns=['ret', 'prccd'])
seasonal_decomposition_factor.to_csv('seasonal_decomposition.csv')